In [22]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("stock.db")
ds_ratio = pd.read_sql_query('select STOCK_CODE, STOCK_FACTOR, STOCK_YEAR, STOCK_DATA from VCSC_FINANCIAL_RATIO', conn)
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

ds_ratio = ds_ratio.loc[ds_ratio['STOCK_FACTOR'] == 'EPS (VND)']
ds_ratio = ds_ratio.loc[ds_ratio['STOCK_YEAR'].str.contains('TTM')==False]
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace('-', '')
ds_ratio['STOCK_DATA'] = ds_ratio['STOCK_DATA'].str.replace(',', '')
ds_ratio['STOCK_DATA'] = pd.to_numeric(ds_ratio['STOCK_DATA'])
ds_ratio['STOCK_YEAR'] = pd.to_numeric(ds_ratio['STOCK_YEAR'])

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '')
ds_price['STOCK_CLOSE'] = pd.to_numeric(ds_price['STOCK_CLOSE'])
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_DATE'].dt.to_period("Y")
ds_price = ds_price.merge(ds_ratio, left_on=['STOCK_CODE', 'STOCK_YEAR'], right_on=['STOCK_CODE', 'STOCK_YEAR'], how='left')
ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_QUARTER']).first()

ds_price['RETURN'] = ds_price['STOCK_CLOSE'].div(ds_price.groupby(level=0)['STOCK_CLOSE'].shift(1))
ds_price['EP'] = ds_price['STOCK_DATA'].div(ds_price['STOCK_CLOSE'])

ds_price['QUANTILES'] = ds_price.groupby('STOCK_YEAR')[['EP']].transform(lambda x: pd.cut(x, bins=10, duplicates='drop'))
#ds_price['QUANTILES'] = pd.qcut(ds_price['EP'],10,labels=list(range(10)))
#ds_price['QUANTILES'] = ds_price.groupby('STOCK_YEAR')['EP'].transform(pd.qcut,10,list(range(10)))
ds_price

STOCK_DATE  STOCK_CLOSE  STOCK_YEAR STOCK_FACTOR  \
STOCK_CODE STOCK_QUARTER                                                    
AAA        2016          2016-12-30        16141        2016    EPS (VND)   
           2017          2017-12-29        24383        2017    EPS (VND)   
           2018          2018-07-02        18250        2018          NaN   
AAM        2011          2011-12-30        13621        2011    EPS (VND)   
           2012          2012-12-28        16498        2012    EPS (VND)   
           2013          2013-12-31        11114        2013    EPS (VND)   
           2014          2014-12-31        12022        2014    EPS (VND)   
           2015          2015-12-31         9923        2015    EPS (VND)   
           2016          2016-12-30         9083        2016    EPS (VND)   
           2017          2017-12-29         9875        2017    EPS (VND)   
           2018          2018-07-02        10800        2018          NaN   
AAV        2018          2018-07-02        16700        2018          NaN   
ABC        2017          2017-12-29        14953        2017    EPS (VND)   
           2018          2018-07-02         7004        2018          NaN   
ABI        2011          2011-12-30         2258        2011    EPS (VND)   
           2012          2012-12-28         2991        2012    EPS (VND)   
           2013          2013-12-31         4954        2013    EPS (VND)   
           2014          2014-12-31         7928        2014    EPS (VND)   
           2015          2015-12-31        11229        2015    EPS (VND)   
           2016          2016-12-30        21812        2016    EPS (VND)   
           2017          2017-12-29        26648        2017    EPS (VND)   
           2018          2018-07-02        23800        2018          NaN   
ABR        2018          2018-07-02         7483        2018          NaN   
ABT        2011          2011-12-30        17603        2011    EPS (VND)   
           2012          2012-12-28        26530        2012    EPS (VND)   
           2013          2013-12-31        28331        2013    EPS (VND)   
           2014          2014-12-31        41565        2014    EPS (VND)   
           2015          2015-12-31        42868        2015    EPS (VND)   
           2016          2016-12-30        41596        2016    EPS (VND)   
           2017          2017-12-29        29112        2017    EPS (VND)   
...                             ...          ...         ...          ...   
WSB        2017          2017-06-30        50253        2017          NaN   
WSS        2011          2011-12-30         3000        2011          NaN   
           2012          2012-12-28         4600        2012          NaN   
           2013          2013-12-31         4500        2013          NaN   
           2014          2014-12-31         5300        2014          NaN   
           2015          2015-12-31         7600        2015          NaN   
           2016          2016-12-30         4300        2016          NaN   
           2017          2017-12-29         4000        2017          NaN   
           2018          2018-07-02         2800        2018          NaN   
WTC        2011          2011-12-30         6300        2011          NaN   
           2012          2012-12-28         5600        2012          NaN   
           2013          2013-12-31         1100        2013          NaN   
           2014          2014-12-31         1800        2014          NaN   
           2015          2015-12-31         1800        2015          NaN   
           2016          2016-07-01         3600        2016          NaN   
X20        2018          2018-07-02        12900        2018          NaN   
XHC        2016          2016-12-30        11230        2016    EPS (VND)   
           2017          2017-06-30        15913        2017    EPS (VND)   
XLV        2018          2018-07-02        11000        2018          NaN   
XMD        2015      